* 카운트 기반과 예측 기반을 모두 사용하는 방법론
* 2014년에 미국 스탠포드대학에서 개발한 단어 임베딩 방법론
* 카운트 기반의 LSA(Latent Semantic Analysis)와 예측 기반의 Word2Vec의 단점을 지적하며 이를 보완한다는 목적으로 나왔고, 실제로도 Word2Vec만큼 뛰어난 성능
* 이 두 가지 전부를 사용해보고 성능이 더 좋은 것을 사용하는 것이 바람직하다.


# 1. 기존 방법론에 대한 비판

## LSA
* LSA는 각 단어의 빈도수를 카운트 한 행렬이라는 전체적인 통계 정보를 입력으로 받아 차원을 축소(Truncated SVD)하여 잠재된 의미를 끌어내는 방법론
* 운트 기반으로 코퍼스의 전체적인 통계 정보를 고려
* 단어 의미의 유추 작업(Analogy task)에는 성능이 떨어진다.

## Word2Vec
* Word2Vec는 실제값과 예측값에 대한 오차를 손실 함수를 통해 줄여나가며 학습하는 예측 기반의 방법론
* 단어 간 유추 작업에는 LSA보다 뛰어남
* 임베딩 벡터가 윈도우 크기 내에서만 주변 단어를 고려하기 때문에 코퍼스의 전체적인 통계 정보를 반영하지 못한다.

# 2. 윈도우 기반 동시 등장 행렬(Window based Co-occurrence Matrix) 
단어의 동시 등장 행렬은 행과 열을 전체 단어 집합의 단어들로 구성하고, i 단어의 윈도우 크기(Window Size) 내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬을 말합니다. 

- I like deep learning
- I like NLP
- I enjoy flying

위의 예시에서 윈도우 사이즈 1이라고 한다면, 아래의 행렬과 같다.


|카운트	|I	|like	|enjoy|	deep|	learning|	NLP	|flying|
|----|---|---|---|---|---|---|---|
|I	|0	|2	|1|	0|	0|	0|	0|
|like	|2	|0|	0|	1|	0|	1|	0|
|enjoy	|1	|0|	0|	0|	0|	0|	1|
|deep	|0	|1|	0|	0|	1|	0|	0|
|learning|0|0|	0|	1|	0|	0|	0|
|NLP	|0	|1|	0|	0|	0|	0|	0|
|flying	|0	|0|	1|	0	|0	|0	|0|


'I'에서 윈도우 사이즈 1이니까 바로 옆의 단어만 참고하여 <u>llike</u>는 'I'기준으로 두번 등장하고 <u>enjoy</u>는 1번 등장한다. 
* **I** <u>like</u> deep learning
* **I** <u>like</u> NLP
* **I** <u>enjoy</u> flying

위 행렬은 행렬을 전치(Transpose)해도 동일한 행렬이 된다는 특징  
-> i 단어의 윈도우 크기 내에서 k 단어가 등장한 빈도는 반대로 k 단어의 윈도우 크기 내에서 i 단어가 등장한 빈도와 동일하기 때문

# 3. 동시 등장 확률(Co-occurrence Probability)

동시 등장 확률 $P(k\ |\ i)$는 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트하고, 특정 단어 i가 등장했을 때 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률

i를 중심 단어(Center Word), k를 주변 단어(Context Word)라고 했을 때, 위에서 배운 동시 등장 행렬에서 중심 단어 i의 행의 모든 값을 더한 값을 분모로 하고 i행 k열의 값을 분자로 한 값  
* i를 ice, steam으로 정하고 k를 water, fashion이라고 했을때 P(solid l ice) / P(solid l steam)를 계산하면두 단어 모두와 동시 등장하는 경우가 많으면 1에 가까운 값이 나오고, 동시 등자하는 경우가 모두 적어도 1에 가까운 값이 나온다.

# GloVe : 임베딩 된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 만드는 것

# 4. 손실 함수(Loss function)
GloVe는 아래와 같은 관계를 가지도록 임베딩 벡터를 설계한다.

$dot\ product(w_{i}\ \tilde{w_{k}}) \approx\ log\ P(k\ |\ i) = log\ P_{ik}$

최종적으로 아래와 같은 손실 함수를 얻어낼 수 있다.  
$Loss\ function = \sum_{m, n=1}^{V}\ f(X_{mn})(w_{m}^{T}\tilde{w_{n}} + b_{m} + \tilde{b_{n}} - logX_{mn})^{2}$

In [3]:
!pip install glove_python_binary

In [12]:
import nltk
nltk.download('punkt')
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sswwd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
# 저자의 경우 윈도우 바탕화면에서 작업하여서 'C:\Users\USER\Desktop\ted_en-20160408.xml'이 해당 파일의 경로.  
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

In [17]:
from glove import Corpus, Glove
corpus = Corpus()
corpus.fit(result,window=5)

# 경사하강법 학습률 0.05, 아웃풋 벡터의 차원 100
glove = Glove(no_components=100, learning_rate=0.05)
# 쓰레드 개수는 4개, 훈련 횟수는 20번, verbose (설명) True
glove.fit(corpus.matrix, epochs=20, no_threads=10, verbose=True)
# 유사도 검색을 위한 행렬의 index 정보 입력
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 10 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [18]:
model_result1 = glove.most_similar("man")
print(model_result1)

[('woman', 0.9639851588239543), ('guy', 0.8726794502997316), ('girl', 0.863441713905972), ('young', 0.8519259457897719)]


## w2v 결과
[('woman', 0.8452662825584412), ('guy', 0.7914076447486877), ('lady', 0.7580448389053345), ('boy', 0.7566693425178528), ('girl', 0.7327544093132019), ('gentleman', 0.7280296087265015), ('soldier', 0.723828911781311), ('poet', 0.6915566921234131), ('kid', 0.6702145338058472), ('surgeon', 0.6474255919456482)]

In [16]:
model_result3=glove.most_similar("university")
print(model_result3)

[('harvard', 0.8820422020162428), ('mit', 0.8432864536485946), ('stanford', 0.8321941194805292), ('cambridge', 0.8303369834037834)]
